 Importing the necessary libraries

In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
from urllib.request import urlopen
import ssl
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

#import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Import and read file

In [2]:
df = pd.read_html("https://www.bcn.cat/estadistica/castella/dades/timm/ipreus/hab2mave/evo/t2mab.htm")[0]
print (type(df))
df.shape

<class 'pandas.core.frame.DataFrame'>


(95, 13)

#Clean dataframe. Keep Neigborhood "barris" and prices year 2019

At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

Rename the column names
Format the date column
Select data only for the city of Barcelona
Select data only year 2019
Eliminate Neighborhood with Null data


In [3]:
df=df.drop([0,1,2,3,4,5,6,7,8,20,82,83,84,85,86,87,88,89,90,91,92,93,94,51],axis=0)
df = df.drop(df.columns[[1,2,3,4,5,6,7,8,9,10,12]], axis=1)
df.columns = ['Barri', 'Year_2019']
#df.dropna(0)
df.drop(df[df.Year_2019=='n.d.'].index, inplace=True)


In [4]:
df['Barri'] = df['Barri'].str.replace(r'\d+','')

In [5]:
df.drop(df[df.Year_2019=='NaN'].index, inplace=True)
#df.dropna()

df.reset_index(drop=True, inplace=True)
df.dropna(0)

,Barri,Year_2019
0,. el Raval,4.591
1,. el Barri Gòtic,3.811
2,. la Barceloneta,4.849
3,". Sant Pere, Santa Caterina i la Ribera",4.772
4,. el Fort Pienc,4.250
5,. la Sagrada Família,4.173
6,. la Dreta de l'Eixample,5.514
7,. l'Antiga Esquerra de l'Eixample,5.197
8,. la Nova Esquerra de l'Eixample,4.634
9,. Sant Antoni,4.412


In [6]:
df.describe()

,Barri,Year_2019
count,69,63
unique,69,62
top,. Navas,3.236
freq,1,2


Get the latitude and the longitude coordinates of each neighborhood.

In [7]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library
!pip install folium
print('Libraries imported.')

Libraries imported.


In [8]:
#geolocator = Nominatim()
#df['city_coord'] = df['Barri'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [9]:
df.head()

,Barri,Year_2019
0,. el Raval,4.591
1,. el Barri Gòtic,3.811
2,. la Barceloneta,4.849
3,". Sant Pere, Santa Caterina i la Ribera",4.772
4,. el Fort Pienc,4.250


In [10]:
np_df = df.as_matrix()
np_df[2]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


array([' . la Barceloneta', '4.849'], dtype=object)

In [11]:
for index, item in df.iterrows():
    direc='Barcelona'+np_df[index][0]
    print(direc)
    

Barcelona . el Raval
Barcelona . el Barri Gòtic
Barcelona . la Barceloneta
Barcelona . Sant Pere, Santa Caterina i la Ribera
Barcelona . el Fort Pienc
Barcelona . la Sagrada Família
Barcelona . la Dreta de l'Eixample
Barcelona . l'Antiga Esquerra de l'Eixample
Barcelona . la Nova Esquerra de l'Eixample
Barcelona . Sant Antoni
Barcelona . el Poble Sec-AEI Parc Montjuïc
Barcelona . la Marina de Port
Barcelona . la Font de la Guatlla
Barcelona . Hostafrancs
Barcelona . la Bordeta
Barcelona . Sants-Badal
Barcelona . Sants
Barcelona . les Corts
Barcelona . la Maternitat i Sant Ramon
Barcelona . Pedralbes
Barcelona . Vallvidrera, el Tibidabo i les Planes
Barcelona . Sarrià
Barcelona . les Tres Torres
Barcelona . Sant Gervasi-la Bonanova
Barcelona . Sant Gervasi-Galvany
Barcelona . el Putxet i el Farró
Barcelona . Vallcarca i els Penitents
Barcelona . el Coll
Barcelona . la Salut
Barcelona . la Vila de Gràcia
Barcelona . el Camp d'en Grassot i Gràcia Nova
Barcelona . el Baix Guinardó
Barcelon

In [12]:
df_coor=[]
   
for index, item in df.iterrows():
    address='Barcelona '+np_df[index][0]
    #print(address)

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    df_coor.append({'Barri':np_df[index][0],'Latitude':latitude,'Longitude':longitude})
    print('The geograpical coordinate of {} are {}, {}.'.format(np_df[index][0],latitude, longitude))
df_coordinates=pd.DataFrame(df_coor)


The geograpical coordinate of  . el Raval are 41.3795176, 2.1683678.
The geograpical coordinate of  . el Barri Gòtic are 41.3833947, 2.1769119.
The geograpical coordinate of  . la Barceloneta are 41.3806533, 2.1899274.
The geograpical coordinate of  . Sant Pere, Santa Caterina i la Ribera are 41.372251, 2.1775315.
The geograpical coordinate of  . el Fort Pienc are 41.3959246, 2.1823245.
The geograpical coordinate of  . la Sagrada Família are 41.4034789, 2.1744103330097055.
The geograpical coordinate of  . la Dreta de l'Eixample are 41.39412395, 2.166470697643847.
The geograpical coordinate of  . l'Antiga Esquerra de l'Eixample are 41.38876465, 2.156597362161013.
The geograpical coordinate of  . la Nova Esquerra de l'Eixample are 41.3828159, 2.1499663437362098.
The geograpical coordinate of  . Sant Antoni are 41.3784116, 2.1617677.
The geograpical coordinate of  . el Poble Sec-AEI Parc Montjuïc are 41.3687898, 2.1631845.
The geograpical coordinate of  . la Marina de Port are 41.3602964,

In [13]:
df_coordinates


,Barri,Latitude,Longitude
0,. el Raval,41.379518,2.168368
1,. el Barri Gòtic,41.383395,2.176912
2,. la Barceloneta,41.380653,2.189927
3,". Sant Pere, Santa Caterina i la Ribera",41.372251,2.177532
4,. el Fort Pienc,41.395925,2.182325
5,. la Sagrada Família,41.403479,2.174410
6,. la Dreta de l'Eixample,41.394124,2.166471
7,. l'Antiga Esquerra de l'Eixample,41.388765,2.156597
8,. la Nova Esquerra de l'Eixample,41.382816,2.149966
9,. Sant Antoni,41.378412,2.161768


Table with barri ( Neigborhood) and coordinates

In [14]:
df_merge = df.join(df_coordinates.set_index('Barri'), on='Barri')
df_merge

,Barri,Year_2019,Latitude,Longitude
0,. el Raval,4.591,41.379518,2.168368
1,. el Barri Gòtic,3.811,41.383395,2.176912
2,. la Barceloneta,4.849,41.380653,2.189927
3,". Sant Pere, Santa Caterina i la Ribera",4.772,41.372251,2.177532
4,. el Fort Pienc,4.250,41.395925,2.182325
5,. la Sagrada Família,4.173,41.403479,2.174410
6,. la Dreta de l'Eixample,5.514,41.394124,2.166471
7,. l'Antiga Esquerra de l'Eixample,5.197,41.388765,2.156597
8,. la Nova Esquerra de l'Eixample,4.634,41.382816,2.149966
9,. Sant Antoni,4.412,41.378412,2.161768


Plot Maps

In [15]:

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library
!pip install folium
print('Libraries imported.')

Libraries imported.


In [16]:
!type -a pip
import folium
from geopy.geocoders import Nominatim
# create map of Toronto using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Barri in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Barri']):
    label = '{}'.format(Barri)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
map_barcelona

pip is /opt/conda/envs/Python36/bin/pip
pip is /opt/conda/bin/pip


In [17]:
CLIENT_ID = 'NAYWBRRIY3P4BBK1CARNPQB2ERTWJACBIABJO1EH4E0UCSAI' # your Foursquare ID
CLIENT_SECRET = 'LYFXV1YWTUMG02US1MDQ0AJ2DVF4JPKV3UZTDUD1EMZZ2OQX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NAYWBRRIY3P4BBK1CARNPQB2ERTWJACBIABJO1EH4E0UCSAI
CLIENT_SECRET:LYFXV1YWTUMG02US1MDQ0AJ2DVF4JPKV3UZTDUD1EMZZ2OQX


In [18]:
neighborhood_latitude = df_merge.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_merge.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_merge.loc[0, 'Barri'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  . el Raval are 41.3795176, 2.1683678.


In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NAYWBRRIY3P4BBK1CARNPQB2ERTWJACBIABJO1EH4E0UCSAI&client_secret=LYFXV1YWTUMG02US1MDQ0AJ2DVF4JPKV3UZTDUD1EMZZ2OQX&v=20180605&ll=41.3795176,2.1683678&radius=500&limit=100'

In [20]:
results = requests.get(url).json()

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Chulapio,Cocktail Bar,41.379264,2.165905
1,La Robadora,Gastropub,41.379500,2.170463
2,Arume,Spanish Restaurant,41.378953,2.166008
3,A Tu Bola,Tapas Restaurant,41.380096,2.169054
4,La Monroe,Spanish Restaurant,41.378795,2.170692


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
barcelona_venues = getNearbyVenues(names=df_merge['Barri'],
                                   latitudes=df_merge['Latitude'],
                                   longitudes=df_merge['Longitude'])

 . el Raval
 . el Barri Gòtic
 . la Barceloneta
 . Sant Pere, Santa Caterina i la Ribera
 . el Fort Pienc
 . la Sagrada Família
 . la Dreta de l'Eixample
 . l'Antiga Esquerra de l'Eixample
 . la Nova Esquerra de l'Eixample
 . Sant Antoni
 . el Poble Sec-AEI Parc Montjuïc
 . la Marina de Port
 . la Font de la Guatlla
 . Hostafrancs
 . la Bordeta
 . Sants-Badal
 . Sants
 . les Corts
 . la Maternitat i Sant Ramon
 . Pedralbes
 . Vallvidrera, el Tibidabo i les Planes
 . Sarrià
 . les Tres Torres
 . Sant Gervasi-la Bonanova
 . Sant Gervasi-Galvany
 . el Putxet i el Farró
 . Vallcarca i els Penitents
 . el Coll
 . la Salut
 . la Vila de Gràcia
 . el Camp d'en Grassot i Gràcia Nova
 . el Baix Guinardó
 . Can Baró
 . el Guinardó
 . la Font d'en Fargues
 . el Carmel
 . la Teixonera
 . Sant Genís dels Agudells ()
 . Montbau ()
 . la Clota ()
 . Vilapicina i la Torre Llobeta
 . Porta
 . el Turó de la Peira ()
 . Can Peguera ()
 . la Guineueta
 . Canyelles ()
 . les Roquetes ()
 . Verdun
 . la Pro

In [26]:
print(barcelona_venues.shape)
barcelona_venues.head()

(2877, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,. el Raval,41.379518,2.168368,Chulapio,41.379264,2.165905,Cocktail Bar
1,. el Raval,41.379518,2.168368,La Robadora,41.379500,2.170463,Gastropub
2,. el Raval,41.379518,2.168368,Arume,41.378953,2.166008,Spanish Restaurant
3,. el Raval,41.379518,2.168368,A Tu Bola,41.380096,2.169054,Tapas Restaurant
4,. el Raval,41.379518,2.168368,La Monroe,41.378795,2.170692,Spanish Restaurant


In [27]:
barcelona_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
. Can Baró,26,26,26,26,26,26
. Can Peguera (),11,11,11,11,11,11
. Canyelles (),4,4,4,4,4,4
. Ciutat Meridiana (),9,9,9,9,9,9
. Diagonal Mar i el Front Marítim del Poblenou,36,36,36,36,36,36
. Hostafrancs,56,56,56,56,56,56
. Montbau (),19,19,19,19,19,19
. Navas,5,5,5,5,5,5
. Pedralbes,20,20,20,20,20,20


In [28]:
print('There are {} uniques categories.'.format(len(barcelona_venues['Venue Category'].unique())))

There are 277 uniques categories.


In [29]:
# one hot encoding
barcelona_onehot = pd.get_dummies(barcelona_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
barcelona_onehot['Neighborhood'] = barcelona_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [barcelona_onehot.columns[-1]] + list(barcelona_onehot.columns[:-1])
barcelona_onehot = barcelona_onehot[fixed_columns]

barcelona_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Vacation Rental,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
barcelona_onehot.shape

(2877, 277)

In [31]:
barcelona_grouped =barcelona_onehot.groupby('Neighborhood').mean().reset_index()
barcelona_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Vacation Rental,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,. Can Baró,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,. Can Peguera (),0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,. Canyelles (),0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,. Ciutat Meridiana (),0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,. Diagonal Mar i el Front Marítim del Poblenou,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,. Hostafrancs,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.017857,0.035714,0.000000,0.000000,0.000000
6,. Montbau (),0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.052632,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,. Navas,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,. Pedralbes,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,. Porta,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
num_top_venues = 5

for hood in barcelona_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = barcelona_grouped[barcelona_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = barcelona_grouped['Neighborhood']

for ind in np.arange(barcelona_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(barcelona_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,. Can Baró,Spanish Restaurant,Scenic Lookout,Chinese Restaurant,Grocery Store,Breakfast Spot,Dessert Shop,Bookstore,Café,Cambodian Restaurant,Tapas Restaurant
1,. Can Peguera (),Park,Hostel,Supermarket,German Restaurant,Tapas Restaurant,Grocery Store,Restaurant,Basketball Stadium,Food & Drink Shop,Plaza
2,. Canyelles (),BBQ Joint,Pizza Place,Soccer Stadium,Mediterranean Restaurant,Women's Store,Farmers Market,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop
3,. Ciutat Meridiana (),Metro Station,Plaza,Optical Shop,Park,Supermarket,Train Station,Grocery Store,Women's Store,Ethiopian Restaurant,Empanada Restaurant
4,. Diagonal Mar i el Front Marítim del Poblenou,Mediterranean Restaurant,Restaurant,Beach Bar,Beach,Hotel,Spanish Restaurant,Café,Athletics & Sports,Football Stadium,Stadium


In [35]:
# set number of clusters
kclusters = 5

barcelona_grouped_clustering = barcelona_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(barcelona_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 2, 3, 1, 1, 1, 1, 1, 3], dtype=int32)

In [36]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
barcelona_merged = df_merge

# merge barcelona_grouped with barcelona_data to add latitude/longitude for each neighborhood
barcelona_merged = df_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Barri')

barcelona_merged.head() # check the last columns!

,Barri,Year_2019,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,. el Raval,4.591,41.379518,2.168368,1.0,Cocktail Bar,Mediterranean Restaurant,Tapas Restaurant,Bar,Spanish Restaurant,Pizza Place,Bookstore,Café,Jazz Club,Beer Bar
1,. el Barri Gòtic,3.811,41.383395,2.176912,1.0,Tapas Restaurant,Ice Cream Shop,Plaza,Spanish Restaurant,Italian Restaurant,Hotel,Bar,Coffee Shop,Dessert Shop,Café
2,. la Barceloneta,4.849,41.380653,2.189927,1.0,Tapas Restaurant,Mediterranean Restaurant,Bar,Paella Restaurant,Spanish Restaurant,Seafood Restaurant,Burger Joint,Restaurant,Pizza Place,Ice Cream Shop
3,". Sant Pere, Santa Caterina i la Ribera",4.772,41.372251,2.177532,1.0,Hostel,Mediterranean Restaurant,Supermarket,Garden,Park,Chinese Restaurant,Bistro,Coffee Shop,Museum,Boat or Ferry
4,. el Fort Pienc,4.250,41.395925,2.182325,1.0,Coffee Shop,Spanish Restaurant,Mediterranean Restaurant,Diner,Restaurant,Yoga Studio,Sushi Restaurant,Supermarket,Stadium,Bistro


Create Map

In [37]:
!type -a pip
import folium
from geopy.geocoders import Nominatim
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(barcelona_merged['Latitude'], barcelona_merged['Longitude'], barcelona_merged['Barri'], barcelona_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
               
        
#map_clusters

#sense degradacio

# create map of Toronto using latitude and longitude values
#map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Barri in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Barri']):
    label = '{}'.format(Barri)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
map_barcelona


pip is /opt/conda/envs/Python36/bin/pip
pip is /opt/conda/bin/pip


In [38]:
barcelona_merged.loc[barcelona_merged['Cluster Labels'] == 0, barcelona_merged.columns[[0] + list(range(5,barcelona_merged.shape[1]))]]

,Barri,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,". Vallvidrera, el Tibidabo i les Planes",Train Station,BBQ Joint,Restaurant,Women's Store,Falafel Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop


Cluster 2

In [39]:
barcelona_merged.loc[barcelona_merged['Cluster Labels'] == 1, barcelona_merged.columns[[0] + list(range(5,barcelona_merged.shape[1]))]]

,Barri,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,. el Raval,Cocktail Bar,Mediterranean Restaurant,Tapas Restaurant,Bar,Spanish Restaurant,Pizza Place,Bookstore,Café,Jazz Club,Beer Bar
1,. el Barri Gòtic,Tapas Restaurant,Ice Cream Shop,Plaza,Spanish Restaurant,Italian Restaurant,Hotel,Bar,Coffee Shop,Dessert Shop,Café
2,. la Barceloneta,Tapas Restaurant,Mediterranean Restaurant,Bar,Paella Restaurant,Spanish Restaurant,Seafood Restaurant,Burger Joint,Restaurant,Pizza Place,Ice Cream Shop
3,". Sant Pere, Santa Caterina i la Ribera",Hostel,Mediterranean Restaurant,Supermarket,Garden,Park,Chinese Restaurant,Bistro,Coffee Shop,Museum,Boat or Ferry
4,. el Fort Pienc,Coffee Shop,Spanish Restaurant,Mediterranean Restaurant,Diner,Restaurant,Yoga Studio,Sushi Restaurant,Supermarket,Stadium,Bistro
5,. la Sagrada Família,Hotel,Burger Joint,Italian Restaurant,Mexican Restaurant,Wine Bar,Plaza,Bakery,Café,Sandwich Place,Mediterranean Restaurant
6,. la Dreta de l'Eixample,Hotel,Boutique,Tapas Restaurant,Restaurant,Mediterranean Restaurant,Bakery,Spanish Restaurant,Hostel,Furniture / Home Store,Clothing Store
7,. l'Antiga Esquerra de l'Eixample,Spanish Restaurant,Hotel,Cocktail Bar,Mediterranean Restaurant,Tapas Restaurant,Sandwich Place,Beer Bar,Japanese Restaurant,Restaurant,Mexican Restaurant
8,. la Nova Esquerra de l'Eixample,Café,Indian Restaurant,Mediterranean Restaurant,Hotel,Tapas Restaurant,Nightclub,Japanese Restaurant,Gym,Bar,Brewery
9,. Sant Antoni,Mediterranean Restaurant,Café,Cocktail Bar,Bar,Tapas Restaurant,Coffee Shop,Spanish Restaurant,Pizza Place,Sandwich Place,Gift Shop


Cluster 3

In [40]:
barcelona_merged.loc[barcelona_merged['Cluster Labels'] == 2, barcelona_merged.columns[[0] + list(range(5,barcelona_merged.shape[1]))]]

,Barri,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,. Canyelles (),BBQ Joint,Pizza Place,Soccer Stadium,Mediterranean Restaurant,Women's Store,Farmers Market,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop


Cluster 4

In [41]:
barcelona_merged.loc[barcelona_merged['Cluster Labels'] == 3, barcelona_merged.columns[[0] + list(range(5,barcelona_merged.shape[1]))]]

,Barri,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,. Vallcarca i els Penitents,Café,Plaza,Hotel,Bakery,Grocery Store,Supermarket,Breakfast Spot,Burger Joint,Gourmet Shop,Park
33,. el Guinardó,Plaza,Breakfast Spot,Café,Tapas Restaurant,Restaurant,Gym,Playground,Grocery Store,Farmers Market,Supermarket
35,. el Carmel,Food Court,Supermarket,Grocery Store,Gym,Coffee Shop,Café,Plaza,Fish & Chips Shop,Flea Market,Filipino Restaurant
37,. Sant Genís dels Agudells (),Tapas Restaurant,Grocery Store,Tennis Court,Farmers Market,Bus Station,Women's Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop
39,. la Clota (),Park,Coffee Shop,Food Court,Supermarket,Café,Grocery Store,Gym,Spanish Restaurant,Hostel,Plaza
41,. Porta,Spanish Restaurant,Supermarket,Grocery Store,Pizza Place,Plaza,Food,Tapas Restaurant,Sandwich Place,Department Store,Big Box Store
42,. el Turó de la Peira (),Tapas Restaurant,Pizza Place,Café,Food,German Restaurant,Plaza,Pub,Restaurant,Park,Department Store
43,. Can Peguera (),Park,Hostel,Supermarket,German Restaurant,Tapas Restaurant,Grocery Store,Restaurant,Basketball Stadium,Food & Drink Shop,Plaza
44,. la Guineueta,Plaza,Grocery Store,Breakfast Spot,Seafood Restaurant,Fast Food Restaurant,Escape Room,Market,Soccer Field,Diner,Bakery
46,. les Roquetes (),Diner,Gym Pool,Supermarket,Music Venue,Falafel Restaurant,General Entertainment,Castle,Baby Store,Grocery Store,Gym


Cluster 5

In [42]:
barcelona_merged.loc[barcelona_merged['Cluster Labels'] == 4, barcelona_merged.columns[[0] + list(range(5,barcelona_merged.shape[1]))]]

,Barri,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,. Torre Baró (),Scenic Lookout,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Women's Store,Donut Shop
